## Find Spark
The way my system is set up, it's not able to directly import pyspark so I had to `pip install findspark` and now I'm able to use it to find my pyspark and initiate it

In [1]:
import findspark
findspark.init()

Now I'm able to import pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

## Start a Spark Session
(the .config(..) part is for windows)

In [3]:
spark = SparkSession.builder.appName('PopularMovies').getOrCreate()

## Read in the Movie Rating Data
The data is stored in two separate csv files
* movies.csv - Movie titles and genres
* ratings.csv - Movie ratings by user

In [4]:
movies = spark.read.csv('ml100/movies.csv', header=True)
ratings = spark.read.csv('ml100/ratings.csv', header=True, inferSchema=True)

## Take a peak at the data
movieId is the link

In [5]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



In [6]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



## Find the Most Rated Movies
* count the occurence of each movie in the ratings dataframe
* order descending by count
* show the top 10 results
* use .cache() to persist the dataframe in memory for quick access
 * useful when accessing data repeatedly

In [7]:
mostRated = ratings.groupBy('movieId').count().orderBy('count', ascending=False).cache()
mostRated.show(10)

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  341|
|    296|  324|
|    318|  311|
|    593|  304|
|    260|  291|
|    480|  274|
|   2571|  259|
|      1|  247|
|    527|  244|
|    589|  237|
+-------+-----+
only showing top 10 rows



## Now Get the Titles for the Top Movies
* left join mostRated with movies by movieId
* sort descending by count
* only show the count and title columns
* only show the top 10

In [8]:
#mostRated.join(movies, mostRated.movieId == movies.movieId).orderBy('count', ascending=False).select('count', 'title').show(10)
mostRated.join(movies, 'movieId').orderBy('count', ascending=False).select('movieId', 'count', 'title').show(10)

+-------+-----+--------------------+
|movieId|count|               title|
+-------+-----+--------------------+
|    356|  341| Forrest Gump (1994)|
|    296|  324| Pulp Fiction (1994)|
|    318|  311|Shawshank Redempt...|
|    593|  304|Silence of the La...|
|    260|  291|Star Wars: Episod...|
|    480|  274|Jurassic Park (1993)|
|   2571|  259|  Matrix, The (1999)|
|      1|  247|    Toy Story (1995)|
|    527|  244|Schindler's List ...|
|    589|  237|Terminator 2: Jud...|
+-------+-----+--------------------+
only showing top 10 rows



## Now get the Average Rating for each Movies

In [9]:
averageRatings = ratings.groupBy('movieId').mean('rating').orderBy('avg(rating)', ascending=False).cache()

In [10]:
averageRatings.show(5)

+-------+-----------+
|movieId|avg(rating)|
+-------+-----------+
|  32460|        5.0|
|  91690|        5.0|
|   5071|        5.0|
|  32525|        5.0|
|  61250|        5.0|
+-------+-----------+
only showing top 5 rows



Right now this doesn't mean much since there are most likely several titles with a 5 star rating but with a count of 1

Let's join this with the mostRated dataframe so we have average rating and count

In [11]:
ratingsCount = mostRated.join(averageRatings, 'movieId').cache()

In [12]:
ratingsCount.show(5)

+-------+-----+------------------+
|movieId|count|       avg(rating)|
+-------+-----+------------------+
|    356|  341|  4.05425219941349|
|    296|  324| 4.256172839506172|
|    318|  311| 4.487138263665595|
|    593|  304|4.1381578947368425|
|    260|  291| 4.221649484536083|
+-------+-----+------------------+
only showing top 5 rows



Now that we have count and average rating together, select criteria for minimum number of ratings (50 here) and then sort by average rating descending

In [13]:
top10 = ratingsCount.where(ratingsCount['count'] >= 50).orderBy('avg(rating)', ascending=False)
top10.show(10)

+-------+-----+------------------+
|movieId|count|       avg(rating)|
+-------+-----+------------------+
|    858|  200|            4.4875|
|    318|  311| 4.487138263665595|
|    969|   50|              4.42|
|    913|   62| 4.387096774193548|
|   1221|  135| 4.385185185185185|
|     50|  201| 4.370646766169155|
|   1228|   50|              4.35|
|   1252|   76|4.3355263157894735|
|    904|   92| 4.315217391304348|
|   1203|   74| 4.304054054054054|
+-------+-----+------------------+
only showing top 10 rows



Now join with the movies dataframe to get the title

In [14]:
top10.join(movies, 'movieId').select('movieId', 'count', 'avg(rating)', 'title').show(10)

+-------+-----+------------------+--------------------+
|movieId|count|       avg(rating)|               title|
+-------+-----+------------------+--------------------+
|    858|  200|            4.4875|Godfather, The (1...|
|    318|  311| 4.487138263665595|Shawshank Redempt...|
|    969|   50|              4.42|African Queen, Th...|
|    913|   62| 4.387096774193548|Maltese Falcon, T...|
|   1221|  135| 4.385185185185185|Godfather: Part I...|
|     50|  201| 4.370646766169155|Usual Suspects, T...|
|   1228|   50|              4.35|  Raging Bull (1980)|
|   1252|   76|4.3355263157894735|    Chinatown (1974)|
|    904|   92| 4.315217391304348|  Rear Window (1954)|
|   1203|   74| 4.304054054054054| 12 Angry Men (1957)|
+-------+-----+------------------+--------------------+
only showing top 10 rows



In [15]:
spark.stop()